<a href="https://colab.research.google.com/github/sean950722/hello/blob/main/%E3%80%8CEX05_04_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

In [ ]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('YOUR_NGROK_AUTHTOKEN'))
    ngrok.connect(5000)
    run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TemplateMessage, ConfirmTemplate,
    MessageAction,
)
app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route("/", methods=['GET'])
def index():
  return "Hello"

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'



@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == 'confirm':
          template = ConfirmTemplate(
            text='你喜歡葬送的芙莉蓮嗎',
            actions=[
              MessageAction(label='是', text='我超喜歡'),
              MessageAction(label='否', text='我不喜歡'),
            ]
          )
          reply=TemplateMessage(
              alt_text="這是確認視窗",
              template=template
          )
        elif action=='carousel':
          carousel_template = CarouselTemplate(
              columns=[
                  CarouselColumn(
                      thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/7/7d/Summer_Wars_poster.jpg',
                      title='Title 1',
                      text='Text 1',
                      actions=[
                          MessageAction(label='Action 1', text='Action 1'),
                          MessageAction(label='Action 2', text='Action 2')
                      ]
                  ),
                  CarouselColumn(
                      thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/0/04/%E9%AD%AF%E9%82%A6%E4%B8%89%E4%B8%96_%E9%AD%AF%E9%82%A6VS%E8%A4%87%E8%A3%BD%E4%BA%BA.jpg',
                      title='Title 1',
                      text='Text 1',
                      actions=[
                          MessageAction(label='Action 1', text='Action 1'),
                          MessageAction(label='Action 2', text='Action 2')
                      ]
                  )
              ]

              )
          reply=TemplateMessage(
              alt_text="這是輪播視窗",
              template=carousel_template
          )

        else:
          reply=TextMessage(
              text='please type "confirm"'
          )
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[reply]
            )
        )

ngrok_start()
if __name__ == "__main__":
    app.run()

SecretNotFoundError: Secret LINE_CHANNEL_ACCESS_TOKEN does not exist.